In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from time import time
%matplotlib inline
seaborn.set()

In [2]:
n, d = 500, 1000
X = np.random.randn(n, d)
y = np.random.randn(n, 1)  # y needs to be of dimension (n, 1), not (n, )

In [3]:
import torch
from torch.autograd import Variable
from torch.nn import Linear, Module, MSELoss
from torch.optim import LBFGS

class LinearRegressionModel(torch.nn.Module):
    def __init__(self, d=100):
        super(LinearRegressionModel, self).__init__()
        self.fc = Linear(d, 1)
    
    def forward(self, x):
        x = self.fc(x)
        return x

In [7]:
X_t = torch.FloatTensor(X)
y_t = torch.FloatTensor(y)

model = LinearRegressionModel(d=d)  # dimension = number of parameters
# define criterion - loss function
criterion = MSELoss()
# define optimizer
optimizer = LBFGS(model.parameters(), max_iter=100, history_size=10)  # change here iteratation number and history size

In [8]:
# define model and gradient descent
def closure():
    Y_pred = model(Variable(X_t))
    loss = criterion(Y_pred, Variable(y_t))
    epoch_loss = loss.data
    optimizer.zero_grad()
    loss.backward()
    return loss
optimizer.step(closure)
output_w = np.array(model.fc.weight.data)

In [9]:
y_est = X @ output_w.T
print(f"Relative error: {np.linalg.norm(y_est-y) / np.linalg.norm(y)}")

Relative error: 0.004284457195612265


In [ ]:
# Send it to GPU (this code has NOT been tested)

X_t = torch.FloatTensor(X).cuda()
y_t = torch.FloatTensor(y).cuda()

model = LinearRegressionModel(d=d)  # dimension = number of parameters
model.cuda()
# define criterion - loss function
criterion = MSELoss()
# define optimizer
optimizer = LBFGS(model.parameters(), max_iter=100, history_size=10)  # change here iteratation number and history size

# define model and gradient descent
def closure():
    Y_pred = model(Variable(X_t))
    loss = criterion(Y_pred, Variable(y_t))
    epoch_loss = loss.data
    optimizer.zero_grad()
    loss.backward()
    return loss
optimizer.step(closure)
output_w = np.array(model.fc.weight.data)